## 自监督学习
框架：MoCo
数据集：mini-Imagenet

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from data_preparation import get_dataloaders
from model import get_resnet18_model, add_fc_layer
from pretraining import MoCo, train_moco
from training_finetuning import train_and_validate, finetune

c:\Users\heyh0\.conda\envs\deeplearning39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\heyh0\.conda\envs\deeplearning39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\heyh0\.conda\envs\deeplearning39\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# 设置参数

In [2]:
# 设置参数
batch_size = 128
num_epochs_pretrain = 200
num_epochs_finetune = 10   #应该是100
num_classes = 100

构造 Data Loader（如果使用tiny-imagenet数据集会比较慢）

In [3]:
# 获取数据加载器
dataset_type = 'mini'  # 可以选择 'tiny' 或 'mini'
imagenet_loader, train_loader_cifar100, test_loader_cifar100 = get_dataloaders(batch_size, data_type=dataset_type)

Files already downloaded and verified
Files already downloaded and verified


保存data loader

In [4]:
# # 保存数据集对象
# torch.save(imagenet_loader.dataset, 'imagenet_loader.pth')
# torch.save(train_loader_cifar100.dataset, 'train_loader_cifar100.pth')
# torch.save(test_loader_cifar100.dataset, 'test_loader_cifar100.pth')

直接导入data loader

In [5]:
# # 重新加载数据集对象的示例代码（注释掉，仅供参考）
# imagenet_dataset = torch.load('imagenet_loader.pth')
# train_dataset_cifar100 = torch.load('train_loader_cifar100.pth')
# test_dataset_cifar100 = torch.load('test_loader_cifar100.pth')
# imagenet_loader = DataLoader(imagenet_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
# train_loader_cifar100 = DataLoader(train_dataset_cifar100, batch_size=batch_size, shuffle=True, num_workers=4)
# test_loader_cifar100 = DataLoader(test_dataset_cifar100, batch_size=batch_size, shuffle=False, num_workers=4)

# MoCo 预训练

In [6]:
# # 这段代码用来测试MoCo的训练是否正常，用最小的数据集测试会快一些
# imagenet_loader = test_loader_cifar100

In [7]:
# MoCo 预训练
print(f"Starting MoCo pre-training on {dataset_type}-ImageNet...")
moco_model = MoCo(base_encoder=get_resnet18_model, dim=128, K=65536, m=0.999, T=0.07, mlp=False).cuda()
moco_optimizer = optim.Adam(moco_model.parameters(), lr=3e-3, weight_decay=1e-4)
moco_criterion = nn.CrossEntropyLoss().cuda()

Starting MoCo pre-training on mini-ImageNet...


c:\Users\heyh0\.conda\envs\deeplearning39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\heyh0\.conda\envs\deeplearning39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
moco_writer = SummaryWriter(log_dir='logs/moco')

for epoch in range(num_epochs_pretrain):
    train_loss = train_moco(imagenet_loader, moco_model, moco_criterion, moco_optimizer, epoch, moco_writer)
    print(f'Epoch [{epoch+1}/{num_epochs_pretrain}], MoCo Loss: {train_loss:.4f}')
moco_writer.close()

# 保存 MoCo 预训练模型权重
torch.save(moco_model.state_dict(), 'moco_pretrained.pth')

In [ ]:
# 重新导入模型权重的语句
# moco_model.load_state_dict(torch.load('moco_pretrained.pth'))

# 微调
只在MoCo预训练模型上微调。此参数将同时用于三个模型

本实验只微调学习率

In [ ]:
# 获取 MoCo 预训练的 encoder_q
moco_pretrained_model = moco_model.encoder_q
moco_pretrained_model.model.fc = nn.Identity()  # 确保最后一层是恒等映射
full_moco_pretrained_model = add_fc_layer(moco_pretrained_model, num_classes)

In [ ]:
learning_rate_list = [1e-3, 3e1]  #注意一下不寻常的学习率

In [ ]:
# 微调和评估 MoCo 预训练模型
print("Finetuning MoCo pre-trained model on CIFAR-100...")
best_lr = finetune(train_loader_cifar100, test_loader_cifar100, full_moco_pretrained_model, learning_rate_list, num_epochs_finetune)
print(f'Best learning rate for fine-tuning: {best_lr}')

# 重新训练模型，可视化训练曲线

训练自监督模型

In [ ]:
# 评估 MoCo 预训练模型
print("Evaluating MoCo pre-trained model on CIFAR-100...")
moco_pretrained_model = moco_model.encoder_q
moco_pretrained_model.model.fc = nn.Identity()  # 确保最后一层是恒等映射
full_moco_pretrained_model = add_fc_layer(moco_pretrained_model, num_classes)

In [ ]:
#可以在这里直接设置学习率
best_lr = 2e-3

In [ ]:
# 使用最佳学习率训练和评估 MoCo 预训练模型
print("Training MoCo pre-trained model on CIFAR-100 with best learning rate...")
finetune_optimizer = optim.Adam([
    {'params': full_moco_pretrained_model[:-1].parameters(), 'lr': best_lr / 10},
    {'params': full_moco_pretrained_model[-1].parameters(), 'lr': best_lr}
])
finetune_writer = SummaryWriter(log_dir='logs/moco_finetune')
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs_finetune):
    train_loss, train_acc, val_loss, val_acc = train_and_validate(train_loader_cifar100, test_loader_cifar100, full_moco_pretrained_model, criterion, finetune_optimizer, epoch, finetune_writer)
    print(f'Epoch [{epoch+1}/{num_epochs_finetune}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
finetune_writer.close()

# 保存 MoCo 预训练并微调后的模型权重
torch.save(full_moco_pretrained_model.state_dict(), 'moco_finetuned.pth')

In [ ]:
# 重新导入模型权重的语句
# full_moco_pretrained_model.load_state_dict(torch.load('moco_finetuned.pth'))

训练监督模型

In [ ]:
# 评估 ImageNet 预训练模型
print("Evaluating ImageNet pre-trained model on CIFAR-100...")
imagenet_pretrained_model = get_resnet18_model(pretrained=True).cuda()
full_imagenet_pretrained_model = add_fc_layer(imagenet_pretrained_model, num_classes)

In [ ]:
#可以在这里直接设置学习率
imagenet_best_lr = 2e-3

In [ ]:
# 使用最佳学习率训练和评估 ImageNet 预训练模型
print("Training ImageNet pre-trained model on CIFAR-100 with best learning rate...")
finetune_optimizer = optim.Adam([
    {'params': full_imagenet_pretrained_model[:-1].parameters(), 'lr': imagenet_best_lr / 10},
    {'params': full_imagenet_pretrained_model[-1].parameters(), 'lr': imagenet_best_lr}
])
imagenet_finetune_writer = SummaryWriter(log_dir='logs/imagenet_finetune')
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs_finetune):
    train_loss, train_acc, val_loss, val_acc = train_and_validate(train_loader_cifar100, test_loader_cifar100, full_imagenet_pretrained_model, criterion, finetune_optimizer, epoch, imagenet_finetune_writer)
    print(f'Epoch [{epoch+1}/{num_epochs_finetune}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
imagenet_finetune_writer.close()

# 保存 ImageNet 预训练并微调后的模型权重
torch.save(full_imagenet_pretrained_model.state_dict(), 'imagenet_finetuned.pth')

In [ ]:
# 重新导入模型权重的语句
# full_imagenet_pretrained_model.load_state_dict(torch.load('imagenet_finetuned.pth'))

训练随机初始化模型

In [ ]:
# 训练和评估随机初始化模型
print("Training randomly initialized model on CIFAR-100...")
random_init_model = get_resnet18_model(pretrained=False).cuda()
full_random_init_model = add_fc_layer(random_init_model, num_classes)

In [ ]:
#可以在这里直接设置学习率
random_best_lr = 2e-3

In [ ]:
# 使用最佳学习率训练和评估随机初始化模型
print("Training randomly initialized model on CIFAR-100 with best learning rate...")
finetune_optimizer = optim.Adam([
    {'params': full_random_init_model.parameters(), 'lr': random_best_lr}
])
random_finetune_writer = SummaryWriter(log_dir='logs/random_finetune')
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs_finetune):
    train_loss, train_acc, val_loss, val_acc = train_and_validate(train_loader_cifar100, test_loader_cifar100, full_random_init_model, criterion, finetune_optimizer, epoch, random_finetune_writer)
    print(f'Epoch [{epoch+1}/{num_epochs_finetune}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
random_finetune_writer.close()

# 保存随机初始化并微调后的模型权重
torch.save(full_random_init_model.state_dict(), 'random_finetuned.pth')

In [ ]:
# 重新导入模型权重的语句
# full_random_init_model.load_state_dict(torch.load('random_finetuned.pth'))